![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [129]:
# Import the numpy and pandas libraries
import numpy as np
import pandas as pd

# Load the datasets
brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Merge the data sets and drop all null values
sports_revenue = pd.merge(brands, finance, on = "product_id")
sports_revenue = pd.merge(sports_revenue, info, on = "product_id")
sports_revenue = pd.merge(sports_revenue, reviews, on = "product_id")
sports_revenue.dropna(inplace = True)

# Display the head of the sports_revenue DataFrame
sports_revenue.head()

,product_id,brand,listing_price,sale_price,discount,revenue,product_name,description,rating,reviews
1,G27341,Adidas,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,"A modern take on adidas sport heritage, tailor...",3.3,24.0
2,CM0081,Adidas,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,These adidas Puka slippers for women's come wi...,2.6,37.0
3,B44832,Adidas,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,"Inspired by modern tech runners, these women's...",4.1,35.0
4,D98205,Adidas,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,This design is inspired by vintage Taekwondo s...,3.5,72.0
5,B75586,Adidas,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,Refine your interval training in these women's...,1.0,45.0


In [130]:
# Add a "price_label" column splitting the listing_price column into labelled quartiles
sports_revenue["price_label"] = pd.qcut(x = sports_revenue["listing_price"], q = 4, labels = ["Budget", "Average", "Expensive", "Elite"])

# Create a new DataFrame grouping by the "brand" and "price_label" columns and aggregate by number of products and mean revenue, rounding all numerical results to two decimal places
adidas_vs_nike = sports_revenue.groupby(by = ["brand", "price_label"], as_index = False).agg(num_products = ("product_id", "count"), mean_revenue = ("revenue", "mean")).round(2)

# Display the adidas_vs_nike DataFrame
adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33
5,Nike,Average,8,675.59
6,Nike,Expensive,47,500.56
7,Nike,Elite,130,1367.45


In [131]:
# Add a "description_length" column splitting the length of the description length into bins of 100 characters
sports_revenue["description_length"] = pd.cut(x = sports_revenue["description"].str.len(), bins = np.arange(0, sports_revenue["description"].str.len().max() + 100, 100)).astype("str")

# Create a new DataFrame grouping by the "description_length" column and aggregate by mean rating and number of reviews, rounding all numerical results to two decimal places
description_lengths = sports_revenue.groupby(by = "description_length", as_index = False).agg(mean_rating = ("rating", "mean"), num_reviews = ("reviews", "count")).round(2)

# Edit the description_length columns of the description_lengths DataFrame
description_lengths["description_length"] = list(map(lambda x: f"{x}", np.arange(100, 800, 100)))

# Display the description_lengths DataFrame
description_lengths

,description_length,mean_rating,num_reviews
0,100,2.26,7
1,200,3.19,526
2,300,3.28,1785
3,400,3.29,651
4,500,3.35,118
5,600,3.12,15
6,700,3.65,15


In [132]:
# Create a regular expression to determine the footwear products and create new DataFrames accordingly
regex = "shoe*|trainer*|foot*"
footwear = sports_revenue[sports_revenue["description"].str.contains(regex)].dropna()
clothing = sports_revenue[~sports_revenue["description"].isin(footwear["description"])].dropna()
# Create a pandas DataFrame aggregating the footwear and clothing DataFrames by number of products and median revenue
product_types = pd.DataFrame({
    "num_clothing_products": clothing["product_id"].count(),
    "median_clothing_revenue": clothing["revenue"].median(),
    "num_footwear_products": footwear["product_id"].count(),
    "median_footwear_revenue": footwear["revenue"].median(),
}, index = [0])

# Display the product_types DataFrame
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,478,625.07,2639,3073.3
